# Imports and Settings

In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import os
from scipy.spatial.distance import squareform, cdist
import time
from numpy.testing import *
from ase.db import connect

In [2]:
from Code.Tools.Data.printer import ProgressTimer
from Code.Tools.Data.saver import create_path
from Code.Tools.Data.transform import get_spherical, change_base, get_input_data

In [4]:
reference_db = "./Dataset/iso17/reference.db"
within_db = "./Dataset/iso17/within.db"
reference_npy = "./Dataset/iso17/reference_"
within_npy = "./Dataset/iso17/within_"
db_paths = [reference_db, within_db]
npy_paths = [reference_npy, within_npy]

In [5]:
for db_path, npy_path in zip(db_paths, npy_paths):
    
    molecules = []
    energies = []
    with connect(db_path) as conn:
        for row in conn.select():
            molecules.append(np.hstack((row['numbers'].reshape((19, 1)), row['positions'])))
            energies.append(row['total_energy'])
    print(np.array(molecules).shape)
    start = time.time()
    network_in = np.array(get_input_data(molecules))
    print('time: {}'.format(time.time() - start))
    np.save(npy_path + 'X', network_in)
    np.save(npy_path + 'Y', energies)

(404000, 19, 4)
0.0  % ---  75.6
0.0  % ---  65.9
0.1  % ---  63.0
0.1  % ---  61.2
0.1  % ---  60.4
0.1  % ---  59.6
0.2  % ---  59.4
0.2  % ---  59.1
0.2  % ---  58.7
0.2  % ---  58.6
0.3  % ---  58.5
0.3  % ---  58.5
0.3  % ---  58.2
0.3  % ---  58.1
0.4  % ---  58.2
0.4  % ---  58.4
0.4  % ---  58.3
0.4  % ---  58.2
0.5  % ---  58.0
0.5  % ---  57.9
0.5  % ---  57.8
0.5  % ---  57.7
0.6  % ---  57.7
0.6  % ---  57.7
0.6  % ---  57.7
0.6  % ---  57.6
0.7  % ---  57.6
0.7  % ---  57.6
0.7  % ---  57.6
0.7  % ---  57.5
0.8  % ---  57.6
0.8  % ---  57.6
0.8  % ---  57.6
0.8  % ---  57.6
0.9  % ---  57.6
0.9  % ---  57.5
0.9  % ---  57.4
0.9  % ---  57.4
1.0  % ---  57.3
1.0  % ---  57.4
1.0  % ---  57.4
1.0  % ---  57.3
1.1  % ---  57.3
1.1  % ---  57.2
1.1  % ---  57.2
1.1  % ---  57.2
1.2  % ---  57.2
1.2  % ---  57.1
1.2  % ---  57.1
1.2  % ---  57.1
1.3  % ---  57.0
1.3  % ---  57.0
1.3  % ---  57.0
1.3  % ---  56.9
1.4  % ---  56.9
1.4  % ---  56.9
1.4  % ---  57.0
1.4  % ---  57.

KeyboardInterrupt: 

# Testing

## Test Functions

In [ ]:
def test_get_spherical():
    test_positions = np.array([[0, 1, 2],
                               [1, 1, 1],
                               [-1, 2, 1]])
    val_result = np.array([[1/np.sqrt(5), np.cos(np.arccos(2/np.sqrt(5))),
                            np.cos(np.pi/2), np.sin(np.pi/2)],
                           [1/np.sqrt(3), np.cos(np.arccos(1/np.sqrt(3))),
                            np.cos(np.arctan(1)), np.sin(np.arctan(1))],
                           [1/np.sqrt(6), np.cos(np.arccos(1/np.sqrt(6))),
                            np.cos(np.arctan(-2) + np.pi), np.sin(np.arctan(-2) + np.pi)]])
    assert_array_almost_equal(val_result, get_spherical(test_positions)) 
    print('pass!')

In [ ]:
def test_change_base():
    test_positions = np.array([[0, 1, 2],
                               [1, 1, 1],
                               [-1, 2, 1]])
    x = np.array([1, 1, 0])
    y = np.array([0, 0, 1])
    z = np.array([2, 1, 3])
    val_result = np.array([[-7.,-13.,4.],
                           [-8.,-17.,5.],
                           [-4., -8.,2.]])
    o = np.array([-1, 4, 3])
    assert_array_almost_equal(val_result, change_base(test_positions, x, y, z, o))
    print('pass!')

In [ ]:
def test_input_data():
    # load molecule and sort
    test_mol = molecules[0]
    test_mol = test_mol[test_mol[:,0].argsort(kind='mergesort')]
    results = np.zeros((19, 72))
    positions = test_mol[:, 1:]
    # H-part
    for i in range(10):
        dists = cdist(positions[np.newaxis, i], positions[10:])
        one, two = dists.argsort().reshape(9)[:2] + 10
        zero = i
        x = positions[one] - positions[zero]
        z = np.cross(x, positions[two] - positions[zero])
        y = np.cross(z, x)
        x /= np.linalg.norm(x)
        y /= np.linalg.norm(y)
        z /= np.linalg.norm(z)
        others = np.vstack((test_mol[:i], test_mol[i+1:]))
        ch_b_others = change_base(others[:, 1:], x, y, z, positions[zero])
        sph_others = get_spherical(ch_b_others)
        ids = sph_others[:, 0].argsort(kind='mergesort')
        sph_others = sph_others[ids]
        others = others[ids]
        sph_others = sph_others[others[:, 0].argsort(kind='mergesort')]
        results[i] = sph_others.reshape(72)
    # C-part
    for i in range(10, 17):
        others = np.vstack((test_mol[:i], test_mol[i+1:]))
        neighs = others[10:]
        dists = cdist(positions[np.newaxis, i], neighs[:, 1:])
        one_id, two_id = dists.argsort().reshape(8)[:2]
        one, two = neighs[one_id, 1:], neighs[two_id, 1:]
        zero =test_mol[i, 1:]
        x = one - zero
        z = np.cross(x, two - zero)
        y = np.cross(z, x)
        x /= np.linalg.norm(x)
        y /= np.linalg.norm(y)
        z /= np.linalg.norm(z)
        ch_b_others = change_base(others[:, 1:], x, y, z, zero)
        sph_others = get_spherical(ch_b_others)
        ids = sph_others[:, 0].argsort(kind='mergesort')
        sph_others = sph_others[ids]
        others = others[ids]
        sph_others = sph_others[others[:, 0].argsort(kind='mergesort')]
        results[i] = sph_others.reshape(72)
    # O-part
    for i in range(17, 19):
        others = np.vstack((test_mol[:i], test_mol[i+1:]))
        neighs = others[10:]
        dists = cdist(positions[np.newaxis, i], neighs[:, 1:])
        one_id, two_id = dists.argsort().reshape(8)[:2]
        one, two = neighs[one_id, 1:], neighs[two_id, 1:]
        zero =test_mol[i, 1:]
        x = one - zero
        z = np.cross(x, two - zero)
        y = np.cross(z, x)
        x /= np.linalg.norm(x)
        y /= np.linalg.norm(y)
        z /= np.linalg.norm(z)
        ch_b_others = change_base(others[:, 1:], x, y, z, zero)
        sph_others = get_spherical(ch_b_others)
        ids = sph_others[:, 0].argsort(kind='mergesort')
        sph_others = sph_others[ids]
        others = others[ids]
        sph_others = sph_others[others[:, 0].argsort(kind='mergesort')]
        results[i] = sph_others.reshape(72)
    test_result = np.array(get_input_data(test_mol[np.newaxis])).reshape(19, 72)
    
    assert_array_almost_equal(test_result, results)
    print('pass!')

## Run Tests

In [ ]:
test_get_spherical()

In [ ]:
test_change_base()

In [ ]:
test_input_data()